In [ ]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import torch
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import numpy as np
from tensorflow.keras.optimizers import Adam
labels = ["ABDOMINAL",
        "ADVANCED-CAD",
        "ALCOHOL-ABUSE",
        "ASP-FOR-MI",
        "CREATININE",
        "DIETSUPP-2MOS",
        "DRUG-ABUSE",
        "ENGLISH",
        "HBA1C",
        "KETO-1YR",
        "MAJOR-DIABETES",
        "MAKES-DECISIONS",
        "MI-6MOS"]


In [ ]:
def get_note_data(labels, folder_name='train', separate=False):
    if separate:
        headers = ["note1", "note2", "note3", "note4", "note5"]
        headers.extend(labels)
        overall_df = pd.DataFrame(columns=headers)
    else:
        headers = ["notes"]
        headers.extend(labels)
        overall_df = pd.DataFrame(columns=headers)

    current_directory = os.getcwd()
    directory = os.path.join(current_directory, folder_name)
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            patient_num = os.path.splitext(filename)[0]
            row_to_add = {}
            # Load the XML file
            tree = ET.parse(os.path.join(directory, filename))
            root = tree.getroot()
            # Access elements and attributes
            for child in root:
                if child.tag == "TEXT":
                    if separate:
                        notes = child.text.split("****************************************************************************************************")
                        notes = [note.strip() for note in notes if note.strip()]
                        i = 1
                        for note in notes:
                            row_to_add[f"note{i}"] = note
                            i += 1
                        for j in range(i, 6):
                            row_to_add[f"note{j}"] = ""
                    else:
                        note = child.text
                        row_to_add['notes'] = note
                if child.tag == "TAGS":
                    for subchild in child:
                        row_to_add[subchild.tag] = 1 if subchild.attrib.get('met') == 'met' else 0
            overall_df.loc[patient_num] = row_to_add

    return overall_df


In [ ]:
patient_notes_data = get_note_data(labels, folder_name='test')
patient_notes_data.sort_index().head()

In [ ]:
def make_model_for_label_combined_notes(df, label, num_epochs, save=False, save_name=None):
    '''
    Makes a fine-tuned ClinicalBERT model for a given label with data.

    Arguments:
    df - DataFrame containing a 'notes' column with the corresponding clinical notes
    label - Str that is the name of the column to use as y/labels for the model training
    save - Flag to save the model
    save_name - What the directory for the model should be named

    Returns:
    TFAutoModelForSequenceClassification
    '''
    if save and save_name is None:
        save_name = f'{label}_model'
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    model = TFAutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", from_pt=True)
    clinical_notes = list(df['notes'])

    # Tokenize and pad the clinical notes
    tokenized_notes = tokenizer(clinical_notes, padding='max_length', max_length=512, truncation=True, return_tensors="tf")
    tokenized_data = dict(tokenized_notes)
    labels = np.array(df[label])

    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=.5, patience=3, monitor='loss')
    model.compile(optimizer=Adam(3e-5), metrics='accuracy')  # No loss argument!

    model.fit(tokenized_data, labels, epochs=num_epochs, verbose=True, callbacks=[lr_scheduler])
    if save:
        tokenizer.save_pretrained(save_name)
        model.save_pretrained(save_name)
    return model



In [ ]:
def make_models_for_labels(labels, make_model_function, df, epochs, save=False):
    '''
    Makes a model for each label using the given function, data, and labels.

    Arguments:
    labels - List[Str] where each string is a label
    make_model_function - Callable to make the model that takes in a df and label
    df - DataFrame of the data
    save - Flag to save each model

    Returns:
    Dict Str -> TFAutoModelForSequenceClassification
    '''
    models = {}
    for label in labels:
        print(f"Making model for label {label}...")
        model = make_model_function(df, label, epochs, save=save)
        print("Finished making model.")
        models[label] = model
    return models

In [ ]:
# Example usage of make_models_for_labels
# models = make_models_for_labels(labels, make_model_for_label_combined_notes, get_note_data(labels), epochs=5, save=True)

In [ ]:
def tokenize_data(note):
    import nltk
    from nltk.corpus import stopwords
    import re
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    pattern = r"[\n\.]"
    # Split the text into sentences
    sentences = re.split(pattern, note.lower())
    # Split the text into tokens
    tokens = [nltk.word_tokenize(sentence) for sentence in sentences if sentence.strip()]
    # Remove stop words
    #filtered_tokens = [token for token in tokens if token not in stop_words]
    return tokens

In [ ]:
loaded_models = {}
for label in labels:
    loaded_models[label] = (TFAutoModelForSequenceClassification.from_pretrained(f'{label}_model'), AutoTokenizer.from_pretrained(f'{label}_model'))
print(loaded_models)
    

In [ ]:
def get_predictions(model, tokenizer, threshold=.5, verbose=True):
    tokenized_notes = tokenizer(list(patient_notes_data["notes"]), padding='max_length', max_length=512, truncation=True, return_tensors="tf")
    tokenized_data = dict(tokenized_notes)
    model_predictions = model.predict(tokenized_data)
    logits = model_predictions.logits
    probabilities = tf.nn.softmax(logits)
    list_probabilities = probabilities.numpy()
    predictions = (list_probabilities[:,0] < threshold).astype(int).tolist()
    return predictions

label_to_predictions = {}
for label, model_tokenizer in loaded_models.items():
    print(f"Predicting for model: f{label}")
    label_to_predictions[label] = get_predictions(model_tokenizer[0], model_tokenizer[1])


In [ ]:
def get_accuracy_from_preds(predictions, true_labels, verbose=True):
    accuracy = np.mean(predictions == true_labels)
    if verbose:
        print(f"{label} Accuracy: {accuracy}")
    return accuracy

label_to_accuracy = {}
for label, predictions in label_to_predictions.items():
    true_labels = np.array(patient_notes_data[label])
    accuracy = get_accuracy_from_preds(predictions, true_labels)
    label_to_accuracy[label] = accuracy

In [ ]:
dfa = pd.DataFrame.from_dict(label_to_accuracy, orient='index', columns=['accuracy'])
dfp = pd.DataFrame.from_dict(label_to_predictions, orient='index')
dfa.to_csv("accuracies.csv")
dfp.to_csv('predictions.csv')

In [ ]:
dfa = pd.read_csv('accuracies.csv')

In [ ]:
dfa.describe()

In [ ]:
# Set the name of the index column
dfp = pd.read_csv('predictions.csv')
label_to_predictions = {}
for index, row in dfp.iterrows():
    label_to_predictions[row[0]] = list(row[1:])